In [46]:
# 각 person_id에 대한 Patient 리소스와 Observation 리소스를 포함하는 Bundle 리소스를
# 자동으로 생성하여 JSON 파일로 저장하는 파이썬 코드

import pandas as pd
import json
import numpy as np
import os

# CSV 파일 경로
new_file_path = 'PFT_transform_test.csv'

# CSV 파일 읽기
df_new = pd.read_csv(new_file_path)

# 모든 환자의 person_id 목록 추출
unique_person_ids = df_new['person_id'].unique()

# JSON 파일을 저장할 디렉토리 생성
output_dir = 'patient_bundles'
os.makedirs(output_dir, exist_ok=True)


In [47]:
import os
import json
import numpy as np
import pandas as pd

# 함수 정의: numpy 타입을 기본 파이썬 타입으로 변환
def convert_numpy(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

# 각 환자에 대해 처리
for person_id in unique_person_ids:
    # 특정 person_id의 데이터 추출
    patient_data = df_new[df_new['person_id'] == person_id]
    
    # Patient 리소스 생성
    patient_resource = {
        "resourceType": "Patient",
        "id": str(person_id),
        "identifier": [
            {
                "use": "official",
                "value": str(person_id)
            }
        ],
        "active": True,
    }

    # Observation 리소스 생성
    observation_resources = []
    for index, row in patient_data.iterrows():
        for column in patient_data.columns[4:]:
            value = row[column]
            if not pd.isna(value):
                observation_resource = {
                    "resourceType": "Observation",
                    "id": f"{column}-{person_id}",
                    "subject": {
                        "reference": f"Patient/{person_id}"
                    },
                    "code": {
                        "coding": [
                            {
                                "system": "http://loinc.org",
                                "code": "19876-2",
                                "display": column
                            }
                        ]
                    },
                    "valueQuantity": {
                        "value": value,
                        "unit": "L",
                        "system": "http://unitsofmeasure.org",
                        "code": "L"
                    }
                }
                observation_resources.append(observation_resource)

    # Bundle 리소스 생성
    bundle_resource = {
        "resourceType": "Bundle",
        "type": "transaction",
        "entry": [
            {
                "fullUrl": f"urn:uuid:{person_id}",
                "resource": patient_resource,
                "request": {
                    "method": "POST",
                    "url": "Patient"
                }
            }
        ] + [
            {
                "fullUrl": f"urn:uuid:{obs['id']}",
                "resource": obs,
                "request": {
                    "method": "POST",
                    "url": "Observation"
                }
            } for obs in observation_resources
        ]
    }

    
    # Bundle 리소스를 JSON 파일로 저장
    bundle_json = json.dumps(bundle_resource, indent=4, default=convert_numpy)
    output_file_path = os.path.join(output_dir, f'patient_{person_id}_bundle.json')
    
    with open(output_file_path, 'w') as file:
        file.write(bundle_json)

# 모든 환자에 대한 처리 완료 후 디렉토리 경로 반환
output_dir


'patient_bundles'

In [48]:
import requests
import json
import numpy as np
import pandas as pd

# Assuming df_new is your DataFrame containing patient data
# And unique_person_ids is a list of unique patient IDs from your DataFrame

# 함수 정의: numpy 타입을 기본 파이썬 타입으로 변환
def convert_numpy(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

# FHIR 서버의 Base URL
url = "http://localhost:8080/fhir"

# 헤더 설정
headers = {
    'Content-Type': 'application/fhir+json'
}

for person_id in unique_person_ids:
    # 특정 person_id의 데이터 추출
    patient_data = df_new[df_new['person_id'] == person_id]
    
    # Patient 리소스 생성
    patient_resource = {
        "resourceType": "Patient",
        "id": str(person_id),
        "identifier": [{"use": "official", "value": str(person_id)}],
        "active": True,
    }

    # Observation 리소스 생성
    observation_resources = []
    for index, row in patient_data.iterrows():
        for column in patient_data.columns[4:]:
            value = row[column]
            if not pd.isna(value):
                observation_resource = {
                    "resourceType": "Observation",
                    "id": f"{column}-{person_id}",
                    "subject": {"reference": f"urn:uuid:{person_id}"},
                    "code": {"coding": [{"system": "http://loinc.org", "code": "19876-2", "display": column}]},
                    "valueQuantity": {"value": value, "unit": "L", "system": "http://unitsofmeasure.org", "code": "L"}
                }
                observation_resources.append(observation_resource)

    # Bundle 리소스 생성
    bundle_resource = {
        "resourceType": "Bundle",
        "type": "transaction",
        "entry": [{
            "fullUrl": f"urn:uuid:{person_id}",
            "resource": patient_resource,
            "request": {"method": "POST", "url": "Patient"}
        }] + [{
            "fullUrl": f"urn:uuid:{obs['id']}",
            "resource": obs,
            "request": {"method": "POST", "url": "Observation"}
        } for obs in observation_resources]
    }

    # Send the bundle to the FHIR server
    response = requests.post(url, json=bundle_resource, headers=headers)
    print(f"Sending Bundle for Patient {person_id}...")
    if response.status_code in [200, 201]:
        print("Success:", response.json())
    else:
        print("Error:", response.status_code, response.text)


Sending Bundle for Patient 1...
Success: {'resourceType': 'Bundle', 'id': '2bb4e5f9-1127-481e-9e17-c7f0ecb7096f', 'type': 'transaction-response', 'link': [{'relation': 'self', 'url': 'http://localhost:8080/fhir'}], 'entry': [{'response': {'status': '201 Created', 'location': 'Patient/21283/_history/1', 'etag': '1', 'lastModified': '2024-04-05T01:52:30.672+00:00', 'outcome': {'resourceType': 'OperationOutcome', 'issue': [{'severity': 'information', 'code': 'informational', 'details': {'coding': [{'system': 'https://hapifhir.io/fhir/CodeSystem/hapi-fhir-storage-response-code', 'code': 'SUCCESSFUL_CREATE', 'display': 'Create succeeded.'}]}, 'diagnostics': 'Successfully created resource "Patient/21283/_history/1". Took 3ms.'}]}}}, {'response': {'status': '201 Created', 'location': 'Observation/21284/_history/1', 'etag': '1', 'lastModified': '2024-04-05T01:52:30.672+00:00', 'outcome': {'resourceType': 'OperationOutcome', 'issue': [{'severity': 'information', 'code': 'informational', 'detail